In [11]:
import numpy as np
import pandas as pd
import os
import time

In [2]:
def count_kmer(seq,k):
    kmer=np.zeros(4**k)
    alphabet = {'A': 0, 'T': 1, 'G': 2, 'C': 3}
    length=len(seq)
    for i in range(length-k+1):
        index=0
        tmp=seq[i:i+k]
        flag=0
        mistake=0
        for j in range(k):
            if tmp[j] != 'A' and tmp[j] != 'T' and tmp[j] != 'C' and tmp[j] != 'G':
                flag=1
                mistake+=1
                break
        if flag==1:
            continue
        for j in range(k):
            index+=(4**(k-j-1))*alphabet[seq[i+j]]
        kmer[index]+=1
    kmer=kmer/(length-k+1-mistake)
    return kmer.tolist()

In [3]:
def read_seq(filename):
    file = open(filename,"r")
    name=[]
    sequence=[]
    seq = ""
    for line in file:
        if line.startswith('>'):
            name.append(line[1:-1])
        else:
            sequence.append(line[:-1])
    file.close()
    return name,sequence

In [4]:
def read_genome(filename):
    file=open(filename,"r")
    name=[]
    sequence=[]
    seq=""
    for line in file:
        if line.startswith('>'):
            if line.find('|')!=-1:
                name.append(line[1:-1].split('|')[3])
        else:
            seq+=line[:-1]
    sequence.append(seq)
    file.close()
    return name,sequence

In [5]:
reflect={0:'Baumannia cicadellinicola str. Hc (Homalodisca coagulata), complete genome',
        1:'Psychromonas ingrahamii 37 chromosome, complete genome',
        2:'Sphingomonas wittichii RW1 chromosome, complete genome',
        3:'Roseiflexus castenholzii DSM 13941 chromosome, complete genome',
        4:'Hydrogenobaculum sp. Y04AAS1 chromosome, complete genome',
        5:'Alteromonas macleodii str. \'Deep ecotype\' chromosome, complete genome',
        6:'Denitrovibrio acetiphilus DSM 12809 chromosome, complete genome',
        7:'Frankia symbiont of Datisca glomerata chromosome, complete genome',
        8:'Candidatus Midichloria mitochondrii IricVA chromosome, complete genome',
        9:'Corynebacterium variabile DSM 44702 chromosome, complete genome'}

In [52]:
def load_genome_seq(genome_dir,k):
    for file in os.listdir(genome_dir):
        print("loading:",file)
        genome_compare=count_kmer(read_genome(genome_dir+file)[1][0],k)
        genome_all_compare.append(genome_compare)
        #name_all.append(read_genome("../DataSource/genomes/"+file)[0][0])
    print("finishied genome")

In [7]:
#pearson
def pearson(k):
    typee=[]
    for i in range(len(num)):
        #print(short_seqs[i])
        short_seq_compare=count_kmer(short_seqs[i],k)
        #print(short_seq_compare)
        score=[]
        for j in range(10):
            #print(np.corrcoef(short_seq_compare,genome_all_compare[j])[1][0])
            score.append(np.corrcoef(short_seq_compare,genome_all_compare[j])[1][0])
        belongto=np.argmax(score)
        typee.append(belongto)
        #print(belongto)
    #print(pd.value_counts(type))
    for i in range(len(num)):
        typee[i]=reflect[typee[i]]
    return typee

In [48]:
#L1范数
def L1(k):
    typeee=[]
    for i in range(len(num)):
        #print(short_seqs[i])
        short_seq_compare=count_kmer(short_seqs[i],k)
        #print(short_seq_compare)
        dist=[]
        for j in range(10):
            #print(np.corrcoef(short_seq_compare,genome_all_compare[j])[1][0]) 
            dist.append(np.sum(np.abs(np.array(short_seq_compare)-np.array(genome_all_compare[j]))))
        belongto=np.argmin(dist)
        typeee.append(belongto)
        #print(belongto)
    #print(pd.value_counts(type))
    for i in range(len(num)):
        typeee[i]=reflect[typeee[i]]
    return typeee

In [29]:
def read_map(filename,species):
    file=open(filename,"r")
    acc=[]
    j=0
    for line in file:
        #print(line[2])
        start=0
        for i in range(10):
            #print(line[i])
            if(line[i]=='B'or line[i]=='P'or line[i]=='S'or line[i]=='R' or line[i]=='H'or line[i]=='A'or line[i]=='D'or line[i]=='F'or line[i]=='C'):
                start=i
            else:
                continue
        #print(start)
        #print(line[start:-1])
        if(line[start:-1]==species[j]):
            acc.append(1)
        else:
            acc.append(0)
        j+=1
    file.close()
    return acc

In [59]:
for i in range(9,10):
    genome_all_compare=[]
    num=[]
    short_seqs=[]
    print("Now,k=%d"%i)
    load_genome_seq("../DataSource/genomes/",i)
    num,short_seqs=read_seq("../DataSource/test.fa")
    print("finishied seqs")
    T1=time.perf_counter()
    type1=pearson(i)
    result1=read_map("../DataSource/seq_id.map",type1)
    accuracy1=pd.value_counts(result1)[1]/len(num)
    T11=time.perf_counter()
        
    T2=time.perf_counter()
    type2=L1(i)
    result2=read_map("../DataSource/seq_id.map",type2)
    accuracy2=pd.value_counts(result2)[1]/len(num)
    T22=time.perf_counter()
        
    fp = open("./Accuracy_test.csv", "a+")  # a+ 如果文件不存在就创建。存在就在文件内容的后面继续追加
    print(i,',',accuracy1,',',(T11-T1)*1000,',',accuracy2,',',(T22-T2)*1000,file=fp)
    fp.close()    

Now,k=9
loading: NC_007984.fna
loading: NC_008709.fna
loading: NC_009511.fna
loading: NC_009767.fna
loading: NC_011126.fna
loading: NC_011138.fna
loading: NC_013943.fna
loading: NC_015656.fna
loading: NC_015722.fna
loading: NC_015859.fna
finishied genome
finishied seqs


- 因为算力不够，分段运行，此处不是全部结果